# Extract graph

In [25]:
from dotenv import load_dotenv
import json
from time import sleep
load_dotenv("../.env")

True

In [5]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.chains import LLMChain
from langchain.llms import BaseLLM
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


In [4]:
chat = ChatOpenAI(model="gpt-3.5-turbo-16k")

Build a chain for knowledge extraction

In [14]:
class KGChain(LLMChain):
    @classmethod
    def from_llm(cls, llm: BaseLLM):

        prompt = PromptTemplate(
            template="""
    Please extract knowledge from the following piece of text from a paper.:
    {paper_text}, current knowledge graph structure is:{knowledge_graph},
    please enhance the knowledge graph with the extracted knowledge if necessary.
    The node types include:
    * disease
    * symptom
    * diagnosis
    * stage
    * test
    * treatment
    * drug
    * gene
    * protein
    The edge types include, but not limited to:
    * has_symptom
    * associate_with
    * diagnose
    * has_stage
    * cure
    if it's references and background research ignore it.
    Please return only the json structure between '###'
    """,
            input_variables=[
                "paper_text",
                "knowledge_graph",
            ],
        )

        return cls(
            llm=llm,
            prompt=prompt,
            verbose=True,
        )

In [10]:
EMPTY_KG_DATA = dict(
    nodes=[
        dict(ntype="disease", name="Esophageal Carcinoma", id=1),
        dict(ntype="symptom", name="Dysphagia", id=2),
    ],
    edges=[
        dict(etype="has_symptom", source=1, target=2),
    ],
)

EMPTY_KG = f"###\n{json.dumps(EMPTY_KG_DATA,)}\n###"

In [15]:
extractor = KGChain.from_llm(llm=chat)

## Data
How to get more data please refer to [crawler](crawler.ipynb)

In [16]:
from pathlib import Path

In [17]:
DATA = Path("./data/")
TEXT = DATA / "text"

In [19]:
paper_list = list(TEXT.glob("*.txt"))
paper_list

[PosixPath('data/text/PMC9713002.txt'),
 PosixPath('data/text/PMC9709273.txt'),
 PosixPath('data/text/PMC9713855.txt'),
 PosixPath('data/text/PMC9708733.txt'),
 PosixPath('data/text/PMC9722938.txt'),
 PosixPath('data/text/PMC9712805.txt'),
 PosixPath('data/text/PMC9713848.txt'),
 PosixPath('data/text/PMC9712015.txt'),
 PosixPath('data/text/PMC9708886.txt'),
 PosixPath('data/text/PMC9714501.txt'),
 PosixPath('data/text/PMC9709130.txt'),
 PosixPath('data/text/PMC9711964.txt'),
 PosixPath('data/text/PMC9713810.txt')]

In [22]:
from tqdm.auto import tqdm
from typing import Iterator

In [23]:
def get_json_string(text, json_end: str = "###") -> str:
    if json_end not in text:
        return text.strip()
    else:
        return text.split(json_end)[1].strip()


def text_slicer(text_path, char_size: int = 3000)->Iterator:
    with open(text_path) as f:
        text = f.read()
    for i in tqdm(list(range(0, len(text), char_size)), leave=False):
        yield text[i:i+char_size]


def extract_one_paper(text_path: Path, char_size: int = 2000 ) -> str:
    kg = EMPTY_KG
    i = 0
    for text in text_slicer(text_path, char_size):
        res = extractor.run(
            paper_text=text,
            knowledge_graph=kg,
        )
        i += 1
        sleep(1)

        try:
            kg = json.dumps(json.loads(get_json_string(res)))
        except json.decoder.JSONDecodeError:
            print(f"Error at {text_path} -({i})")
    return kg
    

In [24]:
kg_string = extract_one_paper(paper_list[0])

  0%|          | 0/17 [00:00<?, ?it/s]



> Entering new  chain...
Prompt after formatting:

    Please extract knowledge from the following piece of text from a paper.:
    Front Med (Lausanne). 2022; 9: 1038097. Published online 2022 Nov 17.  doi: 10.3389/fmed.2022.1038097PMCID: PMC9713002PMID: 36465939Risk factors of lymph node metastasis or lymphovascular invasion for superficial esophageal squamous cell carcinoma: A practical and effective predictive nomogram based on a cancer hospital dataYali Tao,
†
 Shengsen Chen,
†
 Jiangping Yu, Qiwen Shen, Rongwei Ruan,
*
 and  Shi Wang
*
Author information Article notes Copyright and License information PMC DisclaimerDepartment of Endoscopy, Zhejiang Cancer Hospital, Institute of Basic Medicine and Cancer (IBMC), Chinese Academy of Sciences, Hangzhou, ChinaCorresponding author.Edited by: Pietro Mascagni, Fondazione Policlinico Universitario Agostino Gemelli IRCCS, ItalyReviewed by: Zhiyan Liu, Shanghai Jiao Tong University, China; Xiaoying Zhou, First Affiliated Hospital of Nanji

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-BsVT4MvgMqP4ma0WhK2sPKAr on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..



> Finished chain.


> Entering new  chain...
Prompt after formatting:

    Please extract knowledge from the following piece of text from a paper.:
    –), LNM negative and LVI positive (LNM–/LVI+), and LNM positive and LVI positive (LNM+/LVI+) were considered as LNM/LVI positive. LNM/LVI negative meant LNM negative and LVI negative (LNM–/LVI–).Statistical analysisComparisons between continuous variables were performed by using the Mann–Whitney U-test. We compared categorical variables by using the χ2 test or Fisher's exact test. Variables that were significantly associated with LNM/LVI in univariate analysis or had clinical significance for LNM/LVI were incorporated into multivariate logistic regression. LASSO regression is used to reduce the dimensionality of data and screen out the most significant variables with non-zero coefficients (19). The net reclassification improvement (NRI) can be used to assess whether the classification performance of a model will improve if one importan

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-BsVT4MvgMqP4ma0WhK2sPKAr on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-BsVT4MvgMqP4ma0WhK2sPKAr on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. 

...


> Finished chain.
Error at data/text/PMC9713002.txt -(15)


> Entering new  chain...
Prompt after formatting:

    Please extract knowledge from the following piece of text from a paper.:
    nformatics. (2008) 24:2010–4. 10.1093/bioinformatics/btn356 [PubMed] [CrossRef] [Google Scholar]25. Pavey TG, Gilson ND, Gomersall SR, Clark B, Trost SG. Field evaluation of a random forest activity classifier for wrist-worn accelerometer data. J Sci Med Sport. (2017) 20:75–80. 10.1016/j.jsams.2016.06.003 [PubMed] [CrossRef] [Google Scholar]26. Sauer CM, Sasson D, Paik KE, McCague N, Celi LA, Sanchez FI, et al.. Feature selection and prediction of treatment failure in tuberculosis. PLoS ONE. (2018) 13:e207491. 10.1371/journal.pone.0207491 [PMC free article] [PubMed] [CrossRef] [Google Scholar]27. Ishihara R, Iishi H, Uedo N, Takeuchi Y, Yamamoto S, Yamada T, et al.. Comparison of emr and endoscopic submucosal dissection for en bloc resection of early esophageal cancers in japan. Gastrointest Endo

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-BsVT4MvgMqP4ma0WhK2sPKAr on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-BsVT4MvgMqP4ma0WhK2sPKAr on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. 


> Finished chain.
Error at data/text/PMC9713002.txt -(16)


> Entering new  chain...
Prompt after formatting:

    Please extract knowledge from the following piece of text from a paper.:
    4:1–7. 10.1186/s12957-016-1016-0 [PMC free article] [PubMed] [CrossRef] [Google Scholar]36. Update on the paris classification of superficial neoplastic lesions in the digestive tract . Endoscopy. (2005) 37:570–8. 10.1055/s-2005-861352 [PubMed] [CrossRef] [Google Scholar]37. Jia R, Luan Q, Wang J, Hou D, Zhao S. Analysis of predictors for lymph node metastasis in patients with superficial esophageal carcinoma. Gastroent Res Pract. (2016) 2016:1–6. 10.1155/2016/3797615 [PMC free article] [PubMed] [CrossRef] [Google Scholar]38. Iasonos A, Schrag D, Raj GV, Panageas KS. How to build and interpret a nomogram for cancer prognosis. J Clin Oncol. (2008) 26:1364–70. 10.1200/JCO.2007.12.9791 [PubMed] [CrossRef] [Google Scholar]39. Shariat SF, Capitanio U, Jeldres C, Karakiewicz PI. Can nomograms be superi

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-BsVT4MvgMqP4ma0WhK2sPKAr on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-BsVT4MvgMqP4ma0WhK2sPKAr on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. 


> Finished chain.


In [26]:
json.loads(kg_string)

{'nodes': [{'ntype': 'diagnosis',
   'name': 'Lymphovascular Invasion (LVI)',
   'id': 3},
  {'ntype': 'diagnosis', 'name': 'Lymph Node Metastasis (LNM)', 'id': 4},
  {'ntype': 'diagnosis',
   'name': 'Superficial Esophageal Squamous Cell Carcinoma (SESCC)',
   'id': 5},
  {'ntype': 'stage', 'name': 'Tumor Size', 'id': 6},
  {'ntype': 'stage', 'name': 'Circumferential Extension', 'id': 7},
  {'ntype': 'stage', 'name': 'Location within Esophagus', 'id': 8},
  {'ntype': 'stage', 'name': 'Depth of Invasion', 'id': 9},
  {'ntype': 'stage', 'name': 'Tumor Differentiation', 'id': 10},
  {'ntype': 'stage', 'name': 'Macroscopic Type', 'id': 11},
  {'ntype': 'stage', 'name': 'Multiple Lesions', 'id': 12},
  {'ntype': 'test', 'name': 'SPSS', 'id': 13},
  {'ntype': 'test', 'name': 'R', 'id': 14},
  {'ntype': 'treatment', 'name': 'Esophagectomy', 'id': 15},
  {'ntype': 'treatment', 'name': 'Lymph Node Dissection', 'id': 16}],
 'edges': [{'etype': 'associate_with', 'source': 3, 'target': 5},
  {'et

In [34]:
from jinja2 import Template

In [39]:
with open("vis-kg.html") as f:
    nodes_template = Template(f.read())

In [ ]:
# display html

from IPython.display import HTML, display

display(HTML(nodes_template.render(kg_string=kg_string)))